In [1]:
import numpy as np
import pandas as pd
from functools import reduce

In [2]:
with np.load('/media/stasvlad/storage/data/recipes.npz', allow_pickle=True) as data:
    ingredients = data['ingredients']
    recipes = data['recipes']

In [3]:
len(recipes)

1067556

In [4]:
recipes_df = pd.DataFrame(recipes)
recipes_df.head()

,0
0,"[233, 2754, 42, 120, 560, 345, 150, 2081, 12, 21]"
1,"[198, 249, 2, 194, 1884, 791, 965, 423, 53, 48..."
2,"[328, 263, 62, 46, 445, 55, 1196, 82, 664, 3, ..."
3,"[988, 1307]"
4,"[14, 1117, 998, 1010, 9, 30, 18, 29, 214, 1752..."


In [5]:
ingredients_list = ingredients.tolist()
ingredients_map = {ingredient:ingredients_list.index(ingredient) for ingredient in ingredients_list}

In [7]:
df_test = pd.DataFrame(recipes)
df_test.head()

,0
0,"[233, 2754, 42, 120, 560, 345, 150, 2081, 12, 21]"
1,"[198, 249, 2, 194, 1884, 791, 965, 423, 53, 48..."
2,"[328, 263, 62, 46, 445, 55, 1196, 82, 664, 3, ..."
3,"[988, 1307]"
4,"[14, 1117, 998, 1010, 9, 30, 18, 29, 214, 1752..."


In [8]:
df_test = pd.DataFrame(df_test.apply(lambda x: set(x.values[0].tolist()), axis=1))

In [9]:
df_test.head()

,0
0,"{2081, 2754, 233, 42, 12, 560, 21, 150, 120, 345}"
1,"{2, 194, 965, 198, 423, 1031, 8, 362, 48, 94, ..."
2,"{128, 1, 0, 3, 6, 263, 7, 262, 10, 8, 140, 141..."
3,"{1307, 988}"
4,"{1, 132, 1477, 998, 196, 9, 14, 113, 1010, 18,..."


In [10]:
def metric(recipe, shop_list):
    intersection_set = recipe.intersection(shop_list)
    intersection_size = len(intersection_set)
    recipe_size = len(recipe)
    if intersection_size != recipe_size:
        return intersection_size / recipe_size
    else:
        return -1

In [19]:
shopping_list = ['bread', 'eggs']
shopping_list_ids = [ingredients_map[product] for product in shopping_list]
shopping_list_set = set(shopping_list_ids)
shopping_list_set

{13, 69}

In [20]:
%%time
df_test['metric'] = df_test[0].apply(metric, args=(shopping_list_set,))
df_best = df_test.sort_values(by='metric', ascending=False).head(10)
df_best['recommend'] = df_best[0] - shopping_list_set
df_best

CPU times: user 1.25 s, sys: 15 ms, total: 1.26 s
Wall time: 1.29 s


,0,metric,recommend
328992,"{120, 13, 69}",0.666667,{120}
362247,"{13, 69, 130}",0.666667,{130}
676583,"{13, 69, 85}",0.666667,{85}
397741,"{2, 13, 69}",0.666667,{2}
978362,"{13, 69, 766}",0.666667,{766}
406665,"{13, 69, 15}",0.666667,{15}
82190,"{741, 13, 69}",0.666667,{741}
307863,"{13, 69, 71}",0.666667,{71}
300411,"{1683, 13}",0.500000,{1683}
920933,"{285, 13}",0.500000,{285}


In [21]:
%%time
best = list(reduce(set().union, df_best['recommend'].values))
best

CPU times: user 273 µs, sys: 18 µs, total: 291 µs
Wall time: 299 µs


[130, 2, 741, 71, 15, 1683, 85, 120, 285, 766]

In [22]:
ingredients[best]

array(['temperature', 'butter', 'large eggs', 'oil', 'milk', 'eggnog',
       'bacon', 'mozzarella', 'salsa', 'wheat bread'], dtype='<U39')